In [ ]:
import mne
import sys

sys.path.append('..')

from _parameters import *
sys.path.remove('..')

import time
from IPython.display import clear_output


In [ ]:
def run_ica(s: int, n_components = None, auto_pick = False):

    # Load raw
    raw_fname = dirs['raw'] + '/raw_s' + str(s) + '.fif'
    raw = mne.io.read_raw_fif(raw_fname, preload=True)

    # High-pass filter
    raw.filter(1,None)

    # Fit ICA
    ica = mne.preprocessing.ICA(n_components, max_iter="auto").fit(raw)
    
    # EOG correlates
    ica.exclude, scores = ica.find_bads_eog(raw)

    if auto_pick: # Exclude max
        ica.plot_scores(scores)
        ica.plot_components(scores.argmax())
        ica.exclude = [int(scores.argmax())]

    else: # Pick components
        ica.plot_components()
        ica.plot_scores(scores)

        bad_comp = input('Components to exclude (_,_): ').split(sep=',')
        ica.exclude = [int(c) for c in bad_comp]

    # Save
    ica_fname = dirs['ica'] + '/ica_s' + str(s) + '.fif'
    ica.save(ica_fname, overwrite = True)

    return ica

In [ ]:
# Run all subjects

for s in subjects:
    print('Running subject ' + str(s))
    ica = run_ica(s, auto_pick = True)
    time.sleep(5)
    clear_output(wait=False)

In [ ]:
# Run single subject

s = 5

ica = run_ica(s, auto_pick = False)